# Post GWAS analysis Pipeline: Runs of homozygosity using plink
    Description:
    This notebook describe the pipline to screen for runs of homozygous genotypes using Plink 1.9

## Step1: QC
    - annotate samples with gender and phenotype status
    - keep only biallelic snps
    - remove snps with hwe < 1E-5
    - remove snps with missing genotypes > 1%
    - remove snps in high LD

## Software Setup 

### Plink1.9

In [28]:
%%bash

mkdir -p plink_ROH
mkdir -p plink_ROH/Input
mkdir -p plink_ROH/Output
ls
if test -e plink; then

echo "Plink is already installed"
else
echo "Plink is not installed, installing plink....."
cd plink_ROH
wget https://s3.amazonaws.com/plink1-assets/plink_linux_x86_64_20210416.zip 
cd ..
ls plink_ROH
cd ..
unzip plink_ROH/plink_linux_x86_64_20210416.zip
fi


BB_GP2.ipynb
BJ_GP2.ipynb
JL_GP2.ipynb
LICENSE
LW_GP2.ipynb
plink
plink_ROH
prettify
RR_GP2-Copy2.ipynb
RR_GP2.ipynb
RR_GP2_v2.ipynb
toy.map
toy.ped
Plink is already installed


In [1]:
%%bash
#./plink --help
echo "Checking content in plink_ROH directory ....."
ls ./plink_ROH
echo "Checking content in plink_ROH/Input directory ......"
ls ./plink_ROH/Input
echo "Checking content in plink_ROH/Output directory ....."
ls ./plink_ROH/Output


Checking content in plink_ROH directory .....
Input
Output
plink_linux_x86_64_20210416.zip
Checking content in plink_ROH/Input directory ......
all_vcfs.bed
all_vcfs.bim
all_vcfs.fam
all_vcfs.log
all_vcfs_sex_pheno.bed
all_vcfs_sex_pheno.bim
all_vcfs_sex_pheno.fam
all_vcfs_sex_pheno.hh
all_vcfs_sex_pheno.log
all_vcfs_sex_pheno.nosex
all_vcfs_sex_pheno_QC.log
amp_pd_case_control.csv
Demographics.csv
Sample_Gender.tsv
Sample_Pheno.tsv
snp.keep
Checking content in plink_ROH/Output directory .....


## Libray Setup

In [5]:
# Use the os package to interact with the environment
import os

# Use the sys package to interact with stderr
import sys

# Use the FireCloud API to interact with the workspace
from firecloud import api as fapi

# Use Pandas for tabular data
import pandas as pd

# Use StringIO for working with file contents
from io import StringIO

# Enable IPython to display matplotlib graphs.
import matplotlib.pyplot as plt
%matplotlib inline
from cycler import cycler

# Import the iPython HTML rendering for displaying links to Google Cloud Console
from IPython.core.display import display, HTML

# Import urllib modules for building URLs to Google Cloud Console
import urllib.parse

In [7]:
# Utility routine for printing a shell command before executing it
def shell_do(command):
    print(f'Executing: {command}', file=sys.stderr)
    !$command
  
  
def showBfileShape(bfile):
  res = subprocess.run(['wc', '-l', bfile + '.fam'], stdout=subprocess.PIPE)
  N = res.stdout.decode('utf-8').split(' ')[0]
  res = subprocess.run(['wc', '-l', bfile + '.bim'], stdout=subprocess.PIPE)
  V = res.stdout.decode('utf-8').split(' ')[0]
  ans = '=== {} ===\nN = {}, V = {}'.format(bfile, N, V)
  print(ans)
  
  
  # Utility routines for reading files from Google Cloud Storage   
def gcs_read_file(path):
   """Return the contents of a file in GCS"""
   contents = !gsutil -u {BILLING_PROJECT_ID} cat {path}
   return '\n'.join(contents)
  
def gcs_read_csv(path, sep=None):
  """Return a DataFrame from the contents of a delimited file in GCS"""
  return pd.read_csv(StringIO(gcs_read_file(path)), sep=sep, engine='python')
  
  

## Global variable set up

In [6]:
 
  BILLING_PROJECT_ID = os.environ['GOOGLE_PROJECT']
  WORKSPACE_BUCKET = os.environ['WORKSPACE_BUCKET']
  
  GS_WGS_RELEASE_PLINK_PATH = 'gs://amp-pd-genomics/releases/2019_v1release_1015/wgs/plink/bfile'
  
  print(f'GS_WGS_SAMPLES_PATH = {GS_WGS_RELEASE_PLINK_PATH}')
 

GS_WGS_SAMPLES_PATH = gs://amp-pd-genomics/releases/2019_v1release_1015/wgs/plink/bfile


## Copy plink format data to nootebook runtime

In [29]:
shell_do(f'gsutil -mu {BILLING_PROJECT_ID} cp {GS_WGS_RELEASE_PLINK_PATH}/all_vcfs* ./plink_ROH/Input')

shell_do(f'gsutil -mu {BILLING_PROJECT_ID} cp gs://amp-pd-data/releases/2019_v1release_1015/amp_pd_case_control.csv ./plink_ROH/Input')
shell_do(f'gsutil -mu {BILLING_PROJECT_ID} cp gs://amp-pd-data/releases/2019_v1release_1015/clinical/Demographics.csv ./plink_ROH/Input')


Executing: gsutil -mu gp2-ipdgc-hackathon cp gs://amp-pd-genomics/releases/2019_v1release_1015/wgs/plink/bfile/all_vcfs* ./plink_ROH/Input


Copying gs://amp-pd-genomics/releases/2019_v1release_1015/wgs/plink/bfile/all_vcfs.bim...
Copying gs://amp-pd-genomics/releases/2019_v1release_1015/wgs/plink/bfile/all_vcfs.fam...
Copying gs://amp-pd-genomics/releases/2019_v1release_1015/wgs/plink/bfile/all_vcfs.bed...
==> NOTE: You are downloading one or more large file(s), which would            
run significantly faster if you enabled sliced object downloads. This
feature is enabled by default but requires that compiled crcmod be
installed (see "gsutil help crcmod").

Copying gs://amp-pd-genomics/releases/2019_v1release_1015/wgs/plink/bfile/all_vcfs.log...
- [4/4 files][ 21.4 GiB/ 21.4 GiB] 100% Done  85.8 MiB/s ETA 00:00:00           
Operation completed over 4 objects/21.4 GiB.                                     


In [8]:
%%bash
echo "Checking fam file ....."
head -n3 ./plink_ROH/Input/all_vcfs.fam

echo "Checking  bim file....."
head -n3 ./plink_ROH/Input/all_vcfs.bim


Check fam file
BF-1001	BF-1001	0	0	0	-9
BF-1002	BF-1002	0	0	0	-9
BF-1003	BF-1003	0	0	0	-9
Check bim file
1	rs201752861	0	10177	*	A
1	rs201694901	0	10180	C	T
1	rs199706086	0	10250	C	A


## Quality Control of plink data

In [39]:
### add sample gender information
print("checking Demograhics data.....")
demo = pd.read_csv('./plink_ROH/Input/Demographics.csv',sep=',')
demo.head()

checking Demograhics data.....


,participant_id,GUID,visit_name,visit_month,age_at_baseline,sex,ethnicity,race,education_level_years
0,PD-PDNE299YPT,PDNE299YPT,M0,0,74,Male,Not Hispanic or Latino,White,Greater than 16 years
1,PD-PDYW828VAV,PDYW828VAV,M0,0,57,Female,Not Hispanic or Latino,White,12-16 years
2,PD-PDWV958JPG,PDWV958JPG,M0,0,68,Male,Not Hispanic or Latino,White,Greater than 16 years
3,PD-PDRF387ZV3,PDRF387ZV3,M0,0,68,Male,Not Hispanic or Latino,White,12-16 years
4,PD-PDAC268KWV,PDAC268KWV,M0,0,69,Female,Not Hispanic or Latino,White,Greater than 16 years


In [42]:
df=demo[['participant_id','sex']]
df['FID'] = df['participant_id']
df['IID'] = df['participant_id']
df = df[['FID', 'IID', 'sex']]
df['sex'] = df['sex'].astype(str)
df.sex[(df.sex == "Male")] = 1
df.sex[(df.sex == "Female")] = 2
df.columns = ['FID','IID', 'sex']
df.head()


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until


,FID,IID,sex
0,PD-PDNE299YPT,PD-PDNE299YPT,1
1,PD-PDYW828VAV,PD-PDYW828VAV,2
2,PD-PDWV958JPG,PD-PDWV958JPG,1
3,PD-PDRF387ZV3,PD-PDRF387ZV3,1
4,PD-PDAC268KWV,PD-PDAC268KWV,2


In [43]:
with open('./plink_ROH/Input/Sample_Gender.tsv', 'w') as f:
    f.write(df.to_csv(index=False, sep='\t'))

In [46]:
###add case control information
print("checking Phentype data .....")
pheno = pd.read_csv('./plink_ROH/Input/amp_pd_case_control.csv', sep=',')
pheno.head()

checking Phentype data .....


,participant_id,diagnosis_at_baseline,diagnosis_latest,case_control_other_at_baseline,case_control_other_latest
0,BF-1008,Idiopathic PD,Idiopathic PD,Case,Case
1,BF-1156,Idiopathic PD,Idiopathic PD,Case,Case
2,HB-PD_INVAM786LCH,No PD Nor Other Neurological Disorder,No PD Nor Other Neurological Disorder,Control,Control
3,HB-PD_INVBC055WLN,No PD Nor Other Neurological Disorder,No PD Nor Other Neurological Disorder,Control,Control
4,HB-PD_INVBP459TPD,Parkinson's Disease,Parkinson's Disease,Case,Case


In [49]:
df=pheno[['participant_id','case_control_other_latest']]
df['FID'] = df['participant_id']
df['IID'] = df['participant_id']
df['status'] = df['case_control_other_latest']
df = df[['FID', 'IID', 'status']]
df['status'] = df['status'].astype(str)
df.status[(df.status == "Control")] = 1
df.status[(df.status == "Case" )| (df.status == "Other")] = 2
df.columns = ['FID','IID', 'status']
df.head()

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until


,FID,IID,status
0,BF-1008,BF-1008,2
1,BF-1156,BF-1156,2
2,HB-PD_INVAM786LCH,HB-PD_INVAM786LCH,1
3,HB-PD_INVBC055WLN,HB-PD_INVBC055WLN,1
4,HB-PD_INVBP459TPD,HB-PD_INVBP459TPD,2


In [50]:
with open('./plink_ROH/Input/Sample_Pheno.tsv', 'w') as f:
    f.write(df.to_csv(index=False, sep='\t'))

In [53]:
shell_do(f'./plink --bfile ./plink_ROH/Input/all_vcfs \
            --pheno ./plink_ROH/Input/Sample_Pheno.tsv \
            --update-sex ./plink_ROH/Input/Sample_Gender.tsv \
            --make-bed \
            --out ./plink_ROH/Input/all_vcfs_sex_pheno \
            --threads 2')

Executing: ./plink --bfile ./plink_ROH/Input/all_vcfs             --pheno ./plink_ROH/Input/Sample_Pheno.tsv             --update-sex ./plink_ROH/Input/Sample_Gender.tsv             --make-bed             --out ./plink_ROH/Input/all_vcfs_sex_pheno             --threads 2


PLINK v1.90b6.22 64-bit (16 Apr 2021)          www.cog-genomics.org/plink/1.9/
(C) 2005-2021 Shaun Purcell, Christopher Chang   GNU General Public License v3
Logging to ./plink_ROH/Input/all_vcfs_sex_pheno.log.
Options in effect:
  --bfile ./plink_ROH/Input/all_vcfs
  --make-bed
  --out ./plink_ROH/Input/all_vcfs_sex_pheno
  --pheno ./plink_ROH/Input/Sample_Pheno.tsv
  --threads 2
  --update-sex ./plink_ROH/Input/Sample_Gender.tsv

3704 MB RAM detected; reserving 1852 MB for main workspace.
28791969 variants loaded from .bim file.
3074 people (0 males, 0 females, 3074 ambiguous) loaded from .fam.
Ambiguous sex IDs written to ./plink_ROH/Input/all_vcfs_sex_pheno.nosex .
3074 phenotype values present after --pheno.
--update-sex: 3074 people updated, 1225 IDs not present.
Using 1 thread (no multithreaded calculations invoked).
Before main variant filters, 3074 founders and 0 nonfounders present.
Calculating allele frequencies... 101112131415161718192021222324252627282930313233343536373839

In [57]:
%%bash
echo "Checking Gender and phenotype information ....."
head -n3 ./plink_ROH/Input/all_vcfs_sex_pheno.fam


Checking Gender and phenotype information .....
BF-1001 BF-1001 0 0 1 1
BF-1002 BF-1002 0 0 2 2
BF-1003 BF-1003 0 0 1 2


In [65]:
%%bash
head -n3 ./plink_ROH/Input/all_vcfs_sex_pheno.bim


1	rs201752861	0	10177	*	A
1	rs201694901	0	10180	C	T
1	rs199706086	0	10250	C	A


In [67]:
%%bash
###keep biallelic snps
 cat  ./plink_ROH/Input/all_vcfs_sex_pheno.bim | grep -v "*" |awk '{if((length($5)==1) && (length($6)==1)) print }'|cut -f2 > ./plink_ROH/Input/snp.keep
 

In [21]:
shell_do(f'./plink --bfile ./plink_ROH/Input/all_vcfs_sex_pheno \
            --extract ./plink_ROH/Input/snp.keep \
            --make-bed --out ./plink_ROH/Input/all_vcfs_sex_pheno_biallelic')

Executing: ./plink --bfile ./plink_ROH/Input/all_vcfs_sex_pheno             --extract ./plink_ROH/Input/snp.keep             --make-bed --out ./plink_ROH/Input/all_vcfs_sex_pheno_biallelic


PLINK v1.90b6.22 64-bit (16 Apr 2021)          www.cog-genomics.org/plink/1.9/
(C) 2005-2021 Shaun Purcell, Christopher Chang   GNU General Public License v3
Logging to ./plink_ROH/Input/all_vcfs_sex_pheno_biallelic.log.
Options in effect:
  --bfile ./plink_ROH/Input/all_vcfs_sex_pheno
  --extract ./plink_ROH/Input/snp.keep
  --make-bed
  --out ./plink_ROH/Input/all_vcfs_sex_pheno_biallelic

3704 MB RAM detected; reserving 1852 MB for main workspace.
28791969 variants loaded from .bim file.
3074 people (1721 males, 1353 females) loaded from .fam.
3074 phenotype values loaded from .fam.
--extract: 26519563 variants remaining.
Using 1 thread.
Before main variant filters, 3074 founders and 0 nonfounders present.
Calculating allele frequencies... 10111213141516171819202122232425262728293031323334353637383940414243444546474849505152535455565758596061626364656667686970717273747576777879808182838485868788899091929394959697989 done.
./plink_ROH/Input/all_vcfs_sex_pheno_biallelic.hh ); many com

In [23]:
shell_do(f'gsutil -m -u {BILLING_PROJECT_ID} mv ./plink_ROH/Input/all_vcfs_sex_pheno.* {WORKSPACE_BUCKET}')

Executing: gsutil -m -u gp2-ipdgc-hackathon mv ./plink_ROH/Input/all_vcfs_sex_pheno.* gs://fc-secure-90f2e0a1-f2e0-4d16-bdf0-ec7fb247a6d3


Copying file://./plink_ROH/Input/all_vcfs_sex_pheno.bed [Content-Type=application/octet-stream]...
Copying file://./plink_ROH/Input/all_vcfs_sex_pheno.bim [Content-Type=application/octet-stream]...
Copying file://./plink_ROH/Input/all_vcfs_sex_pheno.fam [Content-Type=application/octet-stream]...
Copying file://./plink_ROH/Input/all_vcfs_sex_pheno.hh [Content-Type=text/x-c++hdr]...
Copying file://./plink_ROH/Input/all_vcfs_sex_pheno.log [Content-Type=application/octet-stream]...
==> NOTE: You are uploading one or more large file(s), which would run          
significantly faster if you enable parallel composite uploads. This
feature can be enabled by editing the
"parallel_composite_upload_threshold" value in your .boto
configuration file. However, note that if you do this large files will
be uploaded as `composite objects
<https://cloud.google.com/storage/docs/composite-objects>`_,which
means that any user who downloads such objects will need to have a
compiled crcmod installed (see "gs

In [24]:
%%bash

ls -lh ./plink_ROH/Input/


total 21G
-rw-rw-r-- 1 jupyter-user users  501 May 11 21:56 all_vcfs.log
-rw-rw-r-- 1 jupyter-user users  19G May 12 14:52 all_vcfs_sex_pheno_biallelic.bed
-rw-rw-r-- 1 jupyter-user users 734M May 12 14:52 all_vcfs_sex_pheno_biallelic.bim
-rw-rw-r-- 1 jupyter-user users  91K May 12 14:52 all_vcfs_sex_pheno_biallelic.fam
-rw-rw-r-- 1 jupyter-user users 430M May 12 14:00 all_vcfs_sex_pheno_biallelic.hh
-rw-rw-r-- 1 jupyter-user users 1.4K May 12 14:53 all_vcfs_sex_pheno_biallelic.log
-rw-rw-r-- 1 jupyter-user users    0 May 12 12:47 all_vcfs_sex_pheno_QC.log
-rw-rw-r-- 1 jupyter-user users 114K May 12 01:30 Sample_Gender.tsv
-rw-rw-r-- 1 jupyter-user users 114K May 12 01:44 Sample_Pheno.tsv
-rw-rw-r-- 1 jupyter-user users 288M May 12 03:03 snp.keep


In [26]:
### remove snps in high LD
shell_do(f'./plink --bfile ./plink_ROH/Input/all_vcfs_sex_pheno_biallelic \
            --hwe 0.0001 --maf 0.05 \
            --indep 50 5 1.5 \
            --out ./plink_ROH/Input/all_vcfs_sex_pheno_biallelic.LD')

Executing: ./plink --bfile ./plink_ROH/Input/all_vcfs_sex_pheno_biallelic             --hwe 0.0001 --maf 0.05             --indep 50 5 1.5             --out ./plink_ROH/Input/all_vcfs_sex_pheno_biallelic.LD


PLINK v1.90b6.22 64-bit (16 Apr 2021)          www.cog-genomics.org/plink/1.9/
(C) 2005-2021 Shaun Purcell, Christopher Chang   GNU General Public License v3
Logging to ./plink_ROH/Input/all_vcfs_sex_pheno_biallelic.LD.log.
Options in effect:
  --bfile ./plink_ROH/Input/all_vcfs_sex_pheno_biallelic
  --hwe 0.0001
  --indep 50 5 1.5
  --maf 0.05
  --out ./plink_ROH/Input/all_vcfs_sex_pheno_biallelic.LD

3704 MB RAM detected; reserving 1852 MB for main workspace.
26519563 variants loaded from .bim file.
3074 people (1721 males, 1353 females) loaded from .fam.
3074 phenotype values loaded from .fam.
Using 1 thread.
Before main variant filters, 3074 founders and 0 nonfounders present.
Calculating allele frequencies... 10111213141516171819202122232425262728293031323334353637383940414243444546474849505152535455565758596061626364656667686970717273747576777879808182838485868788899091929394959697989 done.
./plink_ROH/Input/all_vcfs_sex_pheno_biallelic.LD.hh ); many commands treat
these as missi

In [27]:
shell_do(f'./plink --bfile ./plink_ROH/Input/all_vcfs_sex_pheno_biallelic \
            --extract ./plink_ROH/Input/all_vcfs_sex_pheno_biallelic.LD.prune.in \
            --make-bed --out ./plink_ROH/Input/all_vcfs_sex_pheno_biallelic.LD.Prunned')

Executing: ./plink --bfile ./plink_ROH/Input/all_vcfs_sex_pheno_biallelic             --extract ./plink_ROH/Input/all_vcfs_sex_pheno_biallelic.LD.prune.in             --make-bed --out ./plink_ROH/Input/all_vcfs_sex_pheno_biallelic.LD.Prunned


PLINK v1.90b6.22 64-bit (16 Apr 2021)          www.cog-genomics.org/plink/1.9/
(C) 2005-2021 Shaun Purcell, Christopher Chang   GNU General Public License v3
Logging to ./plink_ROH/Input/all_vcfs_sex_pheno_biallelic.LD.Prunned.log.
Options in effect:
  --bfile ./plink_ROH/Input/all_vcfs_sex_pheno_biallelic
  --extract ./plink_ROH/Input/all_vcfs_sex_pheno_biallelic.LD.prune.in
  --make-bed
  --out ./plink_ROH/Input/all_vcfs_sex_pheno_biallelic.LD.Prunned

3704 MB RAM detected; reserving 1852 MB for main workspace.
26519563 variants loaded from .bim file.
3074 people (1721 males, 1353 females) loaded from .fam.
3074 phenotype values loaded from .fam.
--extract: 478835 variants remaining.
Using 1 thread.
Before main variant filters, 3074 founders and 0 nonfounders present.
Calculating allele frequencies... 10111213141516171819202122232425262728293031323334353637383940414243444546474849505152535455565758596061626364656667686970717273747576777879808182838485868788899091929394959697989 done.

In [28]:
%%bash 
ls -lh ./plink_ROH/Input/


total 22G
-rw-rw-r-- 1 jupyter-user users  501 May 11 21:56 all_vcfs.log
-rw-rw-r-- 1 jupyter-user users  19G May 12 14:52 all_vcfs_sex_pheno_biallelic.bed
-rw-rw-r-- 1 jupyter-user users 734M May 12 14:52 all_vcfs_sex_pheno_biallelic.bim
-rw-rw-r-- 1 jupyter-user users  91K May 12 14:52 all_vcfs_sex_pheno_biallelic.fam
-rw-rw-r-- 1 jupyter-user users 430M May 12 14:00 all_vcfs_sex_pheno_biallelic.hh
-rw-rw-r-- 1 jupyter-user users 430M May 12 15:50 all_vcfs_sex_pheno_biallelic.LD.hh
-rw-rw-r-- 1 jupyter-user users 3.1K May 12 16:12 all_vcfs_sex_pheno_biallelic.LD.log
-rw-rw-r-- 1 jupyter-user users 5.0M May 12 16:12 all_vcfs_sex_pheno_biallelic.LD.prune.in
-rw-rw-r-- 1 jupyter-user users  58M May 12 16:12 all_vcfs_sex_pheno_biallelic.LD.prune.out
-rw-rw-r-- 1 jupyter-user users 352M May 12 16:25 all_vcfs_sex_pheno_biallelic.LD.Prunned.bed
-rw-rw-r-- 1 jupyter-user users  14M May 12 16:25 all_vcfs_sex_pheno_biallelic.LD.Prunned.bim
-rw-rw-r-- 1 jupyter-user users  91K May 12 16:25 all_

In [29]:
shell_do(f'./plink --bfile ./plink_ROH/Input/all_vcfs_sex_pheno_biallelic.LD.Prunned \
             --homozyg -homozyg-window-snp 50 \
             --homozyg-snp 50 \
             --homozyg-window-missing 3 \
             --homozyg-kb 100 --homozyg-density 1000 \
             --out  ./plink_ROH/Output/all_vcfs_sex_pheno_biallelic.LD.Prunned.ROH')

Executing: ./plink --bfile ./plink_ROH/Input/all_vcfs_sex_pheno_biallelic.LD.Prunned              --homozyg -homozyg-window-snp 50              --homozyg-snp 50              --homozyg-window-missing 3              --homozyg-kb 100 --homozyg-density 1000              --out  ./plink_ROH/Output/all_vcfs_sex_pheno_biallelic.LD.Prunned.ROH


PLINK v1.90b6.22 64-bit (16 Apr 2021)          www.cog-genomics.org/plink/1.9/
(C) 2005-2021 Shaun Purcell, Christopher Chang   GNU General Public License v3
Logging to ./plink_ROH/Output/all_vcfs_sex_pheno_biallelic.LD.Prunned.ROH.log.
Options in effect:
  --bfile ./plink_ROH/Input/all_vcfs_sex_pheno_biallelic.LD.Prunned
  --homozyg
  --homozyg-density 1000
  --homozyg-kb 100
  --homozyg-snp 50
  --homozyg-window-missing 3
  --homozyg-window-snp 50
  --out ./plink_ROH/Output/all_vcfs_sex_pheno_biallelic.LD.Prunned.ROH

3704 MB RAM detected; reserving 1852 MB for main workspace.
478835 variants loaded from .bim file.
3074 people (1721 males, 1353 females) loaded from .fam.
3074 phenotype values loaded from .fam.
Using 1 thread.
Before main variant filters, 3074 founders and 0 nonfounders present.
Calculating allele frequencies... 10111213141516171819202122232425262728293031323334353637383940414243444546474849505152535455565758596061626364656667686970717273747576777879808182838485868788